In [1]:
import os
import requests
from pprint import pprint 
import pandas as pd

In [2]:
app_key='JMR7VAKJNSXGNVHLP7SP7WNHP'

In [3]:
def req_result(url, params):
    params["key"]=app_key
    params["aggregateHours"]=24
    #params['startDateTime']='2019-06-13T00:00:00'
    #params['endDateTime']='2019-06-13T23:59:00'
    params['unitGroup']='metric'
    params['contentType']='json'
    params['dayStartTime']='0:0:00'
    params['dayEndTime']='0:0:00'
    #params['location']='Aguadilla, PR'
    
    
    res=requests.get(url, params)
    return res.json()

In [4]:
url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history'

In [5]:
dict_weather= req_result(url,{})

In [6]:
pprint(dict_weather)

{'errorCode': 999,
 'executionTime': -1,
 'message': 'Must specify either location list or input datasource',
 'sessionId': ''}


In [7]:
#dict_weather['locations']['Aguadilla, PR']['values'][0]['wspd']

In [8]:
#list(dict_weather['locations'].keys())[0]


In [9]:
df = pd.read_csv('cities_and_dates1_5.csv', sep=',', index_col=0, parse_dates=[1])

In [10]:
df.head()

,fl_date,city_name,wspd,visibility,weathertype,conditions
21092,2019-04-04,"Durango, CO",NaN,NaN,NaN,NaN
21093,2019-04-04,"Eagle, CO",NaN,NaN,NaN,NaN
21094,2019-04-04,"Eau Claire, WI",NaN,NaN,NaN,NaN
21095,2019-04-04,"El Paso, TX",NaN,NaN,NaN,NaN
21096,2019-04-04,"Elmira/Corning, NY",NaN,NaN,NaN,NaN


In [11]:
df['wspd']=""

In [12]:
df['visibility']=""

In [13]:
df['weathertype']=""

In [14]:
df['conditions']=""

In [15]:
df.dtypes

fl_date        datetime64[ns]
city_name              object
wspd                   object
visibility             object
weathertype            object
conditions             object
dtype: object

In [16]:
df.head()

,fl_date,city_name,wspd,visibility,weathertype,conditions
21092,2019-04-04,"Durango, CO",,,,
21093,2019-04-04,"Eagle, CO",,,,
21094,2019-04-04,"Eau Claire, WI",,,,
21095,2019-04-04,"El Paso, TX",,,,
21096,2019-04-04,"Elmira/Corning, NY",,,,


In [17]:
%%time
i=0
for index, row in df.iterrows():
    dict_weather =req_result(url, {'startDateTime':row['fl_date'].strftime('%Y-%m-%d') + 'T00:00:00',
                     'endDateTime':row['fl_date'].strftime('%Y-%m-%d') + 'T23:59:00',
                    'location':row['city_name']})
    try:
        loc_value = list(dict_weather['locations'].keys())[0]


        dict_weather['locations'][loc_value]['values'][0]['wspd']
        df.at[index, 'wspd']=float(dict_weather['locations'][loc_value]['values'][0]['wspd'])
        df.at[index, 'visibility']=float(dict_weather['locations'][loc_value]['values'][0]['visibility'])
        df.at[index, 'weathertype']=dict_weather['locations'][loc_value]['values'][0]['weathertype']
        df.at[index, 'conditions']=dict_weather['locations'][loc_value]['values'][0]['conditions']
    except TypeError:
        pass

Wall time: 2h 27min 36s


In [18]:
df.to_csv('cities_and_dates_weather1_5.csv')

In [19]:
print('Saved')

Saved
